# FBA QUANT Financial Engineering HW 5
KimSooWan(hse07088@snu.ac.kr)
***

In [1196]:
import pandas as pd
import numpy as np
import math
from scipy.optimize import minimize

### Problem 1. 
Reproduce the payer-swaption lattices in the Coursera Lecture. Now experiment with different values of b (but re-calibrating each time so that the spot rate curve remains unchanged) to see how sensitive the swaption price is to the particular value of b that you choose.


Among other things, this should highlight the importance of calibrating models correctly and understanding whether or not your model is appropriate for the problem at hand. For example, we know that option prices are sensitive to volatility and since b is a volatility parameter in the BDT model, it is clearly important to calibrate it accurately. In fact we may wish to choose and calibrate a separate bi for each time period. If we were pricing a swaption where the underlying swap expired after 10 years instead of just 10 months we would also want to consider whether or not a model that cannot, for example, incorporate mean-reversion should be used for such a task.

In [1197]:
def convert_to_Dataframe(array):
    dataframe = pd.DataFrame(array, columns=["t = {}".format(i) for i in range(array.shape[1])])
    dataframe.index.name ='state'
    return dataframe.round(4)

def get_BDT_short_rate(BDT_a=[5.0]*14, BDT_b=[0.005]*14):
    periods = len(BDT_a)
    BDT_short_rate = np.zeros((periods, periods))
    for time in range(0, BDT_short_rate.shape[1]):
        for state in range(0, time+1):
            BDT_short_rate[state,time] = BDT_a[time] * math.exp(BDT_b[time]*state)
    BDT_short_rate = BDT_short_rate/100
    return BDT_short_rate

def get_BDT_elementary_security_price(BDT_short_rate, q=0.5):
    periods = len(BDT_short_rate)+1
    BDT_elementary_security_price = np.zeros((periods, periods))
    BDT_elementary_security_price[0, 0] = 1
    for time in range(1, periods):
        BDT_elementary_security_price[0, time] = (1-q)/(1+BDT_short_rate[0, time-1])*BDT_elementary_security_price[0, time-1]
        BDT_elementary_security_price[time, time] = q/(1+BDT_short_rate[time-1, time-1])*BDT_elementary_security_price[time-1, time-1]
        for state in range(1, time):
            BDT_elementary_security_price[state, time] = (1-q)/(1+BDT_short_rate[state, time-1])*BDT_elementary_security_price[state, time-1] + q/(1+BDT_short_rate[state-1, time-1])*BDT_elementary_security_price[state-1, time-1]
    return BDT_elementary_security_price

def get_BDT_zcb_value(BDT_elementary_security_price):
    zcb_value = BDT_elementary_security_price.sum(axis=0)
    return zcb_value

def get_BDT_spot_rate(BDT_zcb_value):
    periods = len(BDT_zcb_value)
    BDT_spot_rate = BDT_zcb_value.copy()
    BDT_spot_rate[0] = BDT_zcb_value[0] - 1
    for time in range(1, periods):
        BDT_spot_rate[time] = (1/(BDT_zcb_value[time])**(1/time) - 1)*100
    return BDT_spot_rate

def BDT_calibration(market_spot_rate, BDT_b=[0.005]*14, prting_option = True, q=0.5):
    periods = len(BDT_b)
    def objective(BDT_a):   
        BDT_short_rate = get_BDT_short_rate(BDT_a, BDT_b)
        BDT_elementary_security_price = get_BDT_elementary_security_price(BDT_short_rate, q)
        BDT_zcb_value = get_BDT_zcb_value(BDT_elementary_security_price)
        BDT_spot_rate = get_BDT_spot_rate(BDT_zcb_value)
        error = ((market_spot_rate - BDT_spot_rate[1:]) ** 2).sum()
        return error
    
    BDT_a_initial = [5.0]*periods
    BDT_a_bounds = tuple([(0.0, None)]*periods)
    res = minimize(objective, BDT_a_initial, method='slsqp', bounds=BDT_a_bounds)
    callibrated_BDT_a = res.x
    if(prting_option):
        print(res)
    return callibrated_BDT_a
    
market_spot_rate=np.array([7.3, 7.62, 8.1, 8.45, 9.2, 9.64, 10.12, 10.45, 10.75, 11.22, 11.55, 11.92, 12.2, 12.32])
callibrated_BDT_a = BDT_calibration(market_spot_rate)


     fun: 6.2404034583325935e-06
     jac: array([-1.38395358e-05, -1.86926438e-05, -1.86085994e-05,  2.70712250e-05,
       -3.01976575e-05,  1.12193632e-04, -1.61270132e-04,  2.14667471e-04,
       -1.04228027e-04, -6.75420904e-06,  7.46528992e-05, -7.90480182e-05,
        6.65818812e-05,  9.97768336e-06])
 message: 'Optimization terminated successfully'
    nfev: 856
     nit: 57
    njev: 57
  status: 0
 success: True
       x: array([ 7.30000245,  7.92110329,  9.0209628 ,  9.43639379, 12.12792384,
       11.72609567, 12.83572431, 12.58560332, 12.90429614, 15.19493161,
       14.55003109, 15.61604627, 15.1693555 , 13.44412112])


In [1198]:
BDT_short_rate = get_BDT_short_rate(callibrated_BDT_a)
print("BDT short rate lattice is")
convert_to_Dataframe(BDT_short_rate)

BDT short rate lattice is


,t = 0,t = 1,t = 2,t = 3,t = 4,t = 5,t = 6,t = 7,t = 8,t = 9,t = 10,t = 11,t = 12,t = 13
state,,,,,,,,,,,,,,
0,0.073,0.0792,0.0902,0.0944,0.1213,0.1173,0.1284,0.1259,0.1290,0.1519,0.1455,0.1562,0.1517,0.1344
1,0.000,0.0796,0.0907,0.0948,0.1219,0.1178,0.1290,0.1265,0.1297,0.1527,0.1462,0.1569,0.1525,0.1351
2,0.000,0.0000,0.0911,0.0953,0.1225,0.1184,0.1296,0.1271,0.1303,0.1535,0.1470,0.1577,0.1532,0.1358
3,0.000,0.0000,0.0000,0.0958,0.1231,0.1190,0.1303,0.1278,0.1310,0.1542,0.1477,0.1585,0.1540,0.1365
4,0.000,0.0000,0.0000,0.0000,0.1237,0.1196,0.1310,0.1284,0.1316,0.1550,0.1484,0.1593,0.1548,0.1372
5,0.000,0.0000,0.0000,0.0000,0.0000,0.1202,0.1316,0.1290,0.1323,0.1558,0.1492,0.1601,0.1555,0.1378
6,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1323,0.1297,0.1330,0.1566,0.1499,0.1609,0.1563,0.1385
7,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1303,0.1336,0.1574,0.1507,0.1617,0.1571,0.1392
8,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1343,0.1582,0.1514,0.1625,0.1579,0.1399


In [1199]:
BDT_elementary_security_price = get_BDT_elementary_security_price(BDT_short_rate)
print("BDT Elementary security price lattice is")
convert_to_Dataframe(BDT_elementary_security_price)

BDT Elementary security price lattice is


,t = 0,t = 1,t = 2,t = 3,t = 4,t = 5,t = 6,t = 7,t = 8,t = 9,t = 10,t = 11,t = 12,t = 13,t = 14
state,,,,,,,,,,,,,,,
0,1.0,0.466,0.2159,0.0990,0.0452,0.0202,0.0090,0.0040,0.0018,0.0008,0.0003,0.0001,0.0001,0.0000,0.0000
1,0.0,0.466,0.4317,0.2969,0.1808,0.1008,0.0541,0.0280,0.0142,0.0071,0.0034,0.0016,0.0008,0.0004,0.0002
2,0.0,0.000,0.2158,0.2968,0.2711,0.2013,0.1351,0.0838,0.0496,0.0282,0.0153,0.0082,0.0042,0.0022,0.0011
3,0.0,0.000,0.0000,0.0989,0.1806,0.2012,0.1799,0.1394,0.0989,0.0657,0.0407,0.0244,0.0140,0.0079,0.0044
4,0.0,0.000,0.0000,0.0000,0.0451,0.1005,0.1348,0.1392,0.1235,0.0983,0.0710,0.0486,0.0315,0.0197,0.0122
5,0.0,0.000,0.0000,0.0000,0.0000,0.0201,0.0538,0.0834,0.0986,0.0981,0.0850,0.0679,0.0502,0.0354,0.0242
6,0.0,0.000,0.0000,0.0000,0.0000,0.0000,0.0090,0.0277,0.0492,0.0652,0.0706,0.0677,0.0584,0.0470,0.0362
7,0.0,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0040,0.0140,0.0279,0.0403,0.0482,0.0499,0.0468,0.0412
8,0.0,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0018,0.0070,0.0151,0.0240,0.0311,0.0350,0.0359


In [1200]:
BDT_zcb_value = get_BDT_zcb_value(BDT_elementary_security_price)
print("zcb value is")
pd.Series(BDT_zcb_value)[1:].round(4)


zcb value is


1     0.9320
2     0.8634
3     0.7916
4     0.7229
5     0.6440
6     0.5757
7     0.5093
8     0.4515
9     0.3990
10    0.3453
11    0.3005
12    0.2589
13    0.2239
14    0.1966
dtype: float64

In [1201]:
BDT_spot_rate = get_BDT_spot_rate(BDT_zcb_value)
print("\nspot rate is")
print(pd.Series(BDT_spot_rate/100)[1:].round(4))


spot rate is
1     0.0730
2     0.0762
3     0.0810
4     0.0845
5     0.0920
6     0.0964
7     0.1012
8     0.1045
9     0.1075
10    0.1122
11    0.1155
12    0.1192
13    0.1220
14    0.1232
dtype: float64


In [1202]:
def get_BDT_swap_value(BDT_short_rate, q=0.5, swap_fixed_rate = 0.1165, swap_maturity = 10, swaption_expiration = 2):
    BDT_swap_value = np.zeros((swap_maturity, swap_maturity))
    BDT_swap_value[:,swap_maturity-1] = (BDT_short_rate[:swap_maturity,swap_maturity-1]-swap_fixed_rate)/(1+BDT_short_rate[:swap_maturity,swap_maturity-1])
    for time in range(swap_maturity-2, -1, -1):
        for state in range(0, time+1):
            BDT_swap_value[state, time] = (BDT_swap_value[state+1, time+1]*q+BDT_swap_value[state, time+1]*(1-q)+BDT_short_rate[state, time]-swap_fixed_rate)/(1+BDT_short_rate[state, time])
    return BDT_swap_value

def get_BDT_swaption_value(BDT_swap_value, q=0.5, swaption_expiration = 2):
    BDT_swaption_value = np.zeros((swaption_expiration+1, swaption_expiration+1))
    for state in range(0, swaption_expiration+1):
        BDT_swaption_value[state, swaption_expiration] = max(BDT_swap_value[state, swaption_expiration], 0)
    for time in range(swaption_expiration-1, -1, -1):
        for state in range(0, time+1):
            BDT_swaption_value[state, time] = (BDT_swaption_value[state+1, time+1]*q+BDT_swaption_value[state, time+1]*(1-q))/(1+BDT_short_rate[state, time])
    return BDT_swaption_value

def get_swaption_value_by_BDT_b(BDT_b=[0.005]*14):
    BDT_a = BDT_calibration(market_spot_rate, BDT_b, False)
    BDT_short_rate = get_BDT_short_rate(BDT_a, BDT_b)
    BDT_swap_value = get_BDT_swap_value(BDT_short_rate)
    BDT_swaption_value = get_BDT_swaption_value(BDT_swap_value)
    return BDT_swaption_value[0,0]

In [1203]:
BDT_swap_value = get_BDT_swap_value(BDT_short_rate)
print("BDT swap value when BDT b is constantly 0.005 is")
convert_to_Dataframe(BDT_swap_value)

BDT swap value when BDT b is constantly 0.005 is


,t = 0,t = 1,t = 2,t = 3,t = 4,t = 5,t = 6,t = 7,t = 8,t = 9
state,,,,,,,,,,
0,-0.0716,-0.0349,-0.0018,0.0230,0.0461,0.0458,0.0494,0.0431,0.0386,0.0308
1,0.0000,-0.0318,0.0011,0.0257,0.0485,0.0480,0.0513,0.0447,0.0398,0.0314
2,0.0000,0.0000,0.0040,0.0284,0.0510,0.0502,0.0532,0.0462,0.0409,0.0321
3,0.0000,0.0000,0.0000,0.0311,0.0535,0.0524,0.0551,0.0477,0.0420,0.0327
4,0.0000,0.0000,0.0000,0.0000,0.0560,0.0546,0.0570,0.0493,0.0431,0.0333
5,0.0000,0.0000,0.0000,0.0000,0.0000,0.0568,0.0590,0.0508,0.0443,0.0340
6,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0609,0.0524,0.0454,0.0347
7,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0539,0.0466,0.0353
8,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0477,0.0360


In [1204]:
BDT_swaption_value = get_BDT_swaption_value(BDT_swap_value)
print("BDT swaption value when BDT b is constantly 0.005 is")
convert_to_Dataframe(BDT_swaption_value)

BDT swaption value when BDT b is constantly 0.005 is


,t = 0,t = 1,t = 2
state,,,
0,0.0013,0.0005,0.0000
1,0.0000,0.0023,0.0011
2,0.0000,0.0000,0.0040


In [1205]:
print("PV of swaption value when BDT b is constantly 0.005 is", BDT_swaption_value[0,0])

PV of swaption value when BDT b is constantly 0.005 is 0.0013285705989073214


In [1206]:
print("With notional value of 1m,")
benchmark_val = get_swaption_value_by_BDT_b()*1000000
print("PV of swaption value when BDT b is constantly 0.005 is", round(benchmark_val, 1))
test_b_val = [0.0049, 0.0051, 0.0045, 0.0055, 0.004, 0.006]
for test_b in test_b_val:
    test_val = get_swaption_value_by_BDT_b([test_b]*14)*1000000
    print("with", round(((test_b-0.005)/0.005*100),2),"% error in BDT b, PV of swaption value when BDT b is constantly 0.0051 is", round(test_val, 1), "of error", round((test_val-benchmark_val)/benchmark_val*100,2), "%")

With notional value of 1m,
PV of swaption value when BDT b is constantly 0.005 is 1328.6
with -2.0 % error in BDT b, PV of swaption value when BDT b is constantly 0.0051 is 1316.1 of error -0.94 %
with 2.0 % error in BDT b, PV of swaption value when BDT b is constantly 0.0051 is 1341.1 of error 0.94 %
with -10.0 % error in BDT b, PV of swaption value when BDT b is constantly 0.0051 is 1266.2 of error -4.7 %
with 10.0 % error in BDT b, PV of swaption value when BDT b is constantly 0.0051 is 1391.0 of error 4.7 %
with -20.0 % error in BDT b, PV of swaption value when BDT b is constantly 0.0051 is 1203.8 of error -9.39 %
with 20.0 % error in BDT b, PV of swaption value when BDT b is constantly 0.0051 is 1453.4 of error 9.39 %


***
### Problem 2.
Price the payer-swaption in the Coursera Lecture but now assume that it may be exercised at any time, $t∈{2,3,⋯,9}$, and that the fixed rate in the underlying swap contract is now set at 11.65%. If exercised at time t then the first cash flow occurs at t + 1 based on the short rate prevailing at time t. (Such an instrument is called a Bermudan swaption.)

In [1207]:
Bermudan_swaption_expiration = 9
Bermudan_swaption_exerisable = 2
swap_fixed_rate = 0.1165
Bermudan_swaption_value = np.zeros((Bermudan_swaption_expiration+1, Bermudan_swaption_expiration+1))
Bermudan_swaption_exersied = np.zeros((Bermudan_swaption_expiration+1, Bermudan_swaption_expiration+1))
for state in range(0, Bermudan_swaption_expiration+1):
    Bermudan_swaption_value[state, Bermudan_swaption_expiration] = max(BDT_swap_value[state, Bermudan_swaption_expiration], 0)
for time in range(Bermudan_swaption_expiration-1, -1, -1):
    for state in range(0, time+1):
        if(time>=Bermudan_swaption_exerisable):
            Bermudan_swaption_value[state, time] = (Bermudan_swaption_value[state+1, time+1]*q+Bermudan_swaption_value[state, time+1]*(1-q))/(1+BDT_short_rate[state, time])
            if((BDT_short_rate[state, time]-swap_fixed_rate)/(1+BDT_short_rate[state, time])> 0):
                Bermudan_swaption_exersied[state, time] = True
                Bermudan_swaption_value[state, time] += (BDT_short_rate[state, time]-swap_fixed_rate)/(1+BDT_short_rate[state, time])            
        else:
            Bermudan_swaption_value[state, time] = max((Bermudan_swaption_value[state+1, time+1]*q+Bermudan_swaption_value[state, time+1]*(1-q))/(1+BDT_short_rate[state, time]), 0)
print("Bermudan swaption value is")
convert_to_Dataframe(Bermudan_swaption_value)

Bermudan swaption value is


,t = 0,t = 1,t = 2,t = 3,t = 4,t = 5,t = 6,t = 7,t = 8,t = 9
state,,,,,,,,,,
0,0.0369,0.0386,0.0407,0.0432,0.0461,0.0458,0.0494,0.0431,0.0386,0.0308
1,0.0000,0.0405,0.0427,0.0455,0.0485,0.0480,0.0513,0.0447,0.0398,0.0314
2,0.0000,0.0000,0.0448,0.0477,0.0510,0.0502,0.0532,0.0462,0.0409,0.0321
3,0.0000,0.0000,0.0000,0.0500,0.0535,0.0524,0.0551,0.0477,0.0420,0.0327
4,0.0000,0.0000,0.0000,0.0000,0.0560,0.0546,0.0570,0.0493,0.0431,0.0333
5,0.0000,0.0000,0.0000,0.0000,0.0000,0.0568,0.0590,0.0508,0.0443,0.0340
6,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0609,0.0524,0.0454,0.0347
7,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0539,0.0466,0.0353
8,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0477,0.0360


In [1208]:
print("Bermudan swaption is exercised when ")
convert_to_Dataframe(Bermudan_swaption_exersied)

Bermudan swaption is exercised when 


,t = 0,t = 1,t = 2,t = 3,t = 4,t = 5,t = 6,t = 7,t = 8,t = 9
state,,,,,,,,,,
0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [1209]:
print("PV of Bermudan swaption is", Bermudan_swaption_value[0,0])

PV of Bermudan swaption is 0.036883664847478445


***
### Problem 3.
Construct a short lattice for periods (years) 0 through 9 with an initial rate of 6% and with successive rates determined by a multiplicative factor of either u=1.2 or d=.9. Assign the risk-neutral probabilities to be .5.

1. Using this lattice, find the value of a 10-year 6% bond.
2. Suppose this bond can be called by the issuing party at any time after 5 years. (When the bond is called, the face value plus the currently due coupon are paid at that time and the bond is canceled.) What is the fair value of this bond?
3. Use the forward equation to find the spot rate curve for the lattice.

In [1210]:
u=1.2
d=0.9
q=0.5
periods=10
lattice = pd.DataFrame(np.zeros((periods, periods)), columns=["t = {}".format(i) for i in range(periods)])
lattice.index.name ='state'
short_rate_lattice = lattice.copy()
short_rate_lattice.iloc[0,0]=0.06
for time in range(1, short_rate_lattice.shape[1]):
    short_rate_lattice.iloc[0, time]=short_rate_lattice.iloc[0,time-1]*d
    for state in range(1, time+1):
        short_rate_lattice.iloc[state,time]=short_rate_lattice.iloc[state-1,time-1]*u
short_rate_lattice = short_rate_lattice.round(4)
print("Short lattice is")
short_rate_lattice

Short lattice is


,t = 0,t = 1,t = 2,t = 3,t = 4,t = 5,t = 6,t = 7,t = 8,t = 9
state,,,,,,,,,,
0,0.06,0.054,0.0486,0.0437,0.0394,0.0354,0.0319,0.0287,0.0258,0.0232
1,0.00,0.072,0.0648,0.0583,0.0525,0.0472,0.0425,0.0383,0.0344,0.0310
2,0.00,0.000,0.0864,0.0778,0.0700,0.0630,0.0567,0.0510,0.0459,0.0413
3,0.00,0.000,0.0000,0.1037,0.0933,0.0840,0.0756,0.0680,0.0612,0.0551
4,0.00,0.000,0.0000,0.0000,0.1244,0.1120,0.1008,0.0907,0.0816,0.0735
5,0.00,0.000,0.0000,0.0000,0.0000,0.1493,0.1344,0.1209,0.1088,0.0980
6,0.00,0.000,0.0000,0.0000,0.0000,0.0000,0.1792,0.1612,0.1451,0.1306
7,0.00,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2150,0.1935,0.1741
8,0.00,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2580,0.2322


1. Using this lattice, find the value of a 10-year 6% bond.

In [1211]:
bond_maturity = 10
bond_coupon_rate = 0.06
bond_face_value = 100
print("Assume face value of the bond as", bond_face_value)
bond_value = lattice.copy()
bond_value.iloc[:,bond_maturity-1] = (bond_face_value*(1+bond_coupon_rate))/(1+short_rate_lattice.iloc[:,bond_maturity-1])
for time in range(bond_maturity-2, -1, -1):
    for state in range(0, time+1):
        bond_value.iloc[state, time] = (bond_value.iloc[state, time+1]*(1-q)+bond_value.iloc[state+1, time+1]*q+bond_face_value*bond_coupon_rate)/(1+short_rate_lattice.iloc[state, time])
print("\nThe bond value lattice is")
bond_value.round(4)

Assume face value of the bond as 100

The bond value lattice is


,t = 0,t = 1,t = 2,t = 3,t = 4,t = 5,t = 6,t = 7,t = 8,t = 9
state,,,,,,,,,,
0,91.7189,97.4702,102.2283,105.8504,108.2606,109.4671,109.5025,108.4673,106.4581,103.5966
1,0.0000,84.9739,91.2389,96.5429,100.6915,103.5851,105.1819,105.5240,104.7025,102.8128
2,0.0000,0.0000,78.9451,85.7595,91.6511,96.3705,99.7668,101.7801,102.4286,101.7958
3,0.0000,0.0000,0.0000,73.7724,81.2121,87.7629,93.1168,97.0669,99.5132,100.4644
4,0.0000,0.0000,0.0000,0.0000,69.6331,77.8154,85.1532,91.2460,95.8217,98.7424
5,0.0000,0.0000,0.0000,0.0000,0.0000,66.7754,75.9082,84.2273,91.2224,96.5392
6,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,65.5818,75.9933,85.5984,93.7555
7,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,66.6747,78.8885,90.2819
8,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,71.1311,86.0250


In [1212]:
print("\nPV of the bond is", bond_value.iloc[0,0])


PV of the bond is 91.71891447884184


2. Suppose this bond can be called by the issuing party at any time after 5 years. What is the fair value of this bond?

In [1213]:
callable_bond_value = bond_value.copy()
callable_bond_called = lattice.copy()
callable_period = 5
callable_bond_called.iloc[:,bond_maturity-1] = True
for time in range(bond_maturity-2, -1, -1):
    for state in range(0, time+1):
        callable_bond_value.iloc[state, time] = (callable_bond_value.iloc[state, time+1]*(1-q)+callable_bond_value.iloc[state+1, time+1]*q+bond_face_value*bond_coupon_rate)/(1+short_rate_lattice.iloc[state, time])
        strike_value = (bond_face_value*(1+bond_coupon_rate)/(1+short_rate_lattice.iloc[state, time]))
        if(time >= callable_period and callable_bond_value.iloc[state, time] >= strike_value):
            callable_bond_called.iloc[state, time] = True
            callable_bond_value.iloc[state, time] = strike_value
print("The callable bond value lattice is")
callable_bond_value.round(4)

The callable bond value lattice is


,t = 0,t = 1,t = 2,t = 3,t = 4,t = 5,t = 6,t = 7,t = 8,t = 9
state,,,,,,,,,,
0,91.2144,96.5561,100.6142,103.0872,103.7128,102.3759,102.7231,103.0427,103.3340,103.5966
1,0.0000,84.8184,90.9260,95.9210,99.4713,101.2223,101.6787,102.0900,102.4749,102.8128
2,0.0000,0.0000,78.9246,85.7149,91.5550,96.1649,99.3296,100.8563,101.3481,101.7958
3,0.0000,0.0000,0.0000,73.7724,81.2121,87.7629,93.1168,97.0669,99.5132,100.4644
4,0.0000,0.0000,0.0000,0.0000,69.6331,77.8154,85.1532,91.2460,95.8217,98.7424
5,0.0000,0.0000,0.0000,0.0000,0.0000,66.7754,75.9082,84.2273,91.2224,96.5392
6,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,65.5818,75.9933,85.5984,93.7555
7,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,66.6747,78.8885,90.2819
8,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,71.1311,86.0250


In [1214]:
print("\nWhen does the callable bond is called")
callable_bond_called


When does the callable bond is called


,t = 0,t = 1,t = 2,t = 3,t = 4,t = 5,t = 6,t = 7,t = 8,t = 9
state,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,True,True,True,True,True
1,0.0,0.0,0.0,0.0,0.0,True,True,True,True,True
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,True,True
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True


In [1215]:
print("\nPV of the callable bond is", callable_bond_value.iloc[0,0].round(4))


PV of the callable bond is 91.2144


3. Use the forward equation to find the spot rate curve for the lattice.

In [1216]:
elementary_security_price = lattice.copy()
elementary_security_price.iloc[0, 0] = 1
for time in range(1, periods):
    elementary_security_price.iloc[0, time] = (1-q)/(1+short_rate_lattice.iloc[0, time-1])*elementary_security_price.iloc[0, time-1]
    elementary_security_price.iloc[time, time] = q/(1+short_rate_lattice.iloc[time-1, time-1])*elementary_security_price.iloc[time-1, time-1]
    for state in range(1, time):
        elementary_security_price.iloc[state, time] = (1-q)/(1+short_rate_lattice.iloc[state, time-1])*elementary_security_price.iloc[state, time-1] + q/(1+short_rate_lattice.iloc[state-1, time-1])*elementary_security_price.iloc[state-1, time-1]
print("Elementary_security_price lattice is")
elementary_security_price.round(4)

Elementary_security_price lattice is


,t = 0,t = 1,t = 2,t = 3,t = 4,t = 5,t = 6,t = 7,t = 8,t = 9
state,,,,,,,,,,
0,1.0,0.4717,0.2238,0.1067,0.0511,0.0246,0.0119,0.0058,0.0028,0.0014
1,0.0,0.4717,0.4438,0.3151,0.2000,0.1196,0.0690,0.0388,0.0215,0.0118
2,0.0,0.0000,0.2200,0.3096,0.2925,0.2317,0.1661,0.1117,0.0718,0.0447
3,0.0,0.0000,0.0000,0.1013,0.1895,0.2234,0.2120,0.1771,0.1361,0.0984
4,0.0,0.0000,0.0000,0.0000,0.0459,0.1071,0.1512,0.1672,0.1596,0.1379
5,0.0,0.0000,0.0000,0.0000,0.0000,0.0204,0.0570,0.0938,0.1185,0.1272
6,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0089,0.0289,0.0543,0.0771
7,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0038,0.0140,0.0296
8,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0015,0.0065


In [ ]:
zcb_value = elementary_security_price.sum(axis=0)
print("\nzcb value is")
print(zcb_value.round(4))

spot_rate = zcb_value.copy()
spot_rate.iloc[0] = zcb_value.iloc[0] - 1
for time in range(1, periods):
    spot_rate.iloc[time] = 1/(zcb_value.iloc[time])**(1/time) - 1
print("\nspot rate is")
print(spot_rate.round(4))


zcb value is
t = 0    1.0000
t = 1    0.9434
t = 2    0.8875
t = 3    0.8327
t = 4    0.7790
t = 5    0.7267
t = 6    0.6760
t = 7    0.6270
t = 8    0.5801
t = 9    0.5351
dtype: float64

spot rate is
t = 0    0.0000
t = 1    0.0600
t = 2    0.0615
t = 3    0.0629
t = 4    0.0644
t = 5    0.0659
t = 6    0.0674
t = 7    0.0689
t = 8    0.0704
t = 9    0.0719
dtype: float64


***
### Problem 4.
What is the expected number of cards that need to be turned over in a regular 52-card deck in order to see the first ace?

sol)

The probability of the first ace to be turned over at i-th turn is $$\frac{_{48}\mathrm{C}_{i-1}\times _{4}\mathrm{C}_{1}}{_{52}\mathrm{C}_{i}}$$ thus, expected number of cards to be turen over is as follows $$\sum_{i=1}^{49}i\frac{_{48}\mathrm{C}_{i-1}\times{_4}\mathrm{C}_{1}}{_{52}\mathrm{C}_{i}}=\sum^{49}_{i=1}i\frac{\binom{48}{i-1}}{\binom{52}{i-1}}\frac{4}{52-(i-1)}=10.6$$

In [ ]:
EX = 0
for i in range(1, 50):
    temp = i*(math.comb(48, i-1))/(math.comb(52,i-1))*4/(52-(i-1))
    EX += temp

EX